# GreenValueNet

This notebook contains the code needed to execute the GreenValueNet hedonic pricing neural network. 

### Set up and data loading

In [1]:
import joblib
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.metrics import mean_squared_error
from data_load_funcs import get_params, load_data_catalogue
from processing_funcs import process_data, normalise_values
from model_funcs import *

cwd = Path.cwd()
params = get_params()
data_catalogue = load_data_catalogue()

The `dataset.csv` file from `data/interim_files` is loaded if it already exists, or constructed if not. To build the dataset please follow the steps in [Annex 1 of the ReadMe file](readme.md#annex-1) to esnure all relevant files have been downloaded. 

In [2]:
dataset = process_data(data_catalogue, params)

# show summary stats
summary_stats = dataset.describe().transpose()[['mean', 'std', 'max', 'min']]
summary_stats.columns = ['Mean', 'Std Dev', 'Maximum', 'Minimum']
print(summary_stats)

Processed dataset already exists.
Loading exisitng dataset...
                                  Mean    Std Dev       Maximum   Minimum
ln_price                     12.393713   0.658136     18.134158  0.116855
propertytype                  1.328094   1.085676      3.000000  0.000000
oldnew                        0.134435   0.341119      1.000000  0.000000
duration                      0.325202   0.468482      2.000000  0.000000
current_energy_efficiency    61.056443  12.444568    347.000000  0.000000
potential_energy_efficiency   0.805024   0.140106      1.000000  0.000000
total_floor_area             91.622326  68.669650  60506.700000  0.000000
extension_count               0.495053   0.743755      4.000000  0.000000
number_habitable_rooms        4.521529   1.748808    100.000000  0.000000
number_heated_rooms           4.437662   1.754007    100.000000  0.000000
construction_age_band         4.477500   3.153767     10.000000 -1.000000
coastline_dist               57.173176  38.518186 

In [3]:
print(dataset.shape)
print(dataset.isnull().sum())

(12909200, 26)
transactionid                  0
ln_price                       0
postcode                       0
propertytype                   0
oldnew                         0
duration                       0
current_energy_efficiency      0
potential_energy_efficiency    0
total_floor_area               0
extension_count                0
number_habitable_rooms         0
number_heated_rooms            0
construction_age_band          0
coastline_dist                 0
prim_school_dist               0
sec_school_dist                0
roads_dist                     0
nat_park_dist                  0
nat_trust_dist                 0
ttwa_dist                      0
dom_builds_share               0
garden_share                   0
non_dom_builds_share           0
path_share                     0
greenspace_share               0
water_share                    0
dtype: int64


Now we normalise any non-encoded variables to increase speed of learning of algorithm and convert the dataset to an array of inputs, and an associated output array.

In [4]:
norm_cols = [col for col in dataset.columns if col not in params['non_norm_cols']]
for col in norm_cols:
    dataset[col] = normalise_values(dataset[col])

# creates an input array of shape m,x and an output array of shape m
x, y, derivative_index = create_x_y_arr(dataset, params)

The dataset is then split into train, dev and test sets using sci-kit learn.

In [5]:
x_train, x_dev, x_test, y_train, y_dev, y_test = split_to_test_dev_train(
    x,
    y,
    params['dev_size'],
    params['test_size'],
    prop=False
)

## Benchmarking

To evaluate the performance of my neural network we use random forest and XGBoost regressions as baseline models. We optimise based on the mean squared error (MSE) and report this as our measure of performance.

### Random Forest


In [ ]:
# run baseline random forest regression using scikit-learn
rfr_model = random_forest_reg(
    x_train,
    y_train,
    tuning=False
)

# now run with grid search to tune parameters
# rfr_tuned  =  random_forest_reg(
#     x_train,
#     y_train,
#     tuning=True,
#     tuning_params = params['tuning_dict']['grid']
# )

# generate predictions and measure according to mean squared error
rfr_pred, rfr_mse = generate_pred_metric(rfr_model, mean_squared_error, x_dev, y_dev)
# rfr_t_pred, rfr_t_mse = generate_pred_metric(rfr_tuned, mean_squared_error, x_dev, y_dev)

### XGBoost

In [6]:
xgb_model = boosted_grad_reg(x_train, y_train)
xgb_pred, xgb_mse = generate_pred_metric(xgb_model, mean_squared_error, x_dev, y_dev)

## Neural networks

We know build some neural networks. Number of epochs, hidden layers, and nodes in hidden layers is initially set with rules of thumb but then optimiszed using hyperparameter tuning.

In [7]:
# set epochs to be 3 times number of features
epochs = int(x_train.shape[1]) * 3

# set n_hidden_units to be mean of input and output layer sizes
n_hidden_units = round((x_train.shape[1] + 1) / 2)

### Single Layer Neural Network

A single hidden layer with ReLU activation is used with a linear output layer.

In [8]:
single_nn = neural_net(
    x_train,
    y_train,
    learning_rate = 0.05,
    n_hidden_units = n_hidden_units,
    epochs = epochs,
    validation_data = (x_dev, y_dev)
)

Epoch 1/69
402788/402788 [==============================] - 978s 2ms/step - loss: 0.2600 - mae: 0.3883 - val_loss: 0.2569 - val_mae: 0.3709
Epoch 2/69
402788/402788 [==============================] - 973s 2ms/step - loss: 0.2654 - mae: 0.3961 - val_loss: 0.2694 - val_mae: 0.3829
Epoch 3/69
402788/402788 [==============================] - 965s 2ms/step - loss: 0.2677 - mae: 0.3979 - val_loss: 0.2665 - val_mae: 0.3831
Epoch 4/69
402788/402788 [==============================] - 931s 2ms/step - loss: 0.2677 - mae: 0.3978 - val_loss: 0.2720 - val_mae: 0.3830
Epoch 5/69
402788/402788 [==============================] - 954s 2ms/step - loss: 0.2676 - mae: 0.3977 - val_loss: 0.2772 - val_mae: 0.4019
Epoch 6/69
402788/402788 [==============================] - 1175s 3ms/step - loss: 0.2677 - mae: 0.3978 - val_loss: 0.3001 - val_mae: 0.4272
Epoch 7/69
402788/402788 [==============================] - 821s 2ms/step - loss: 0.2676 - mae: 0.3977 - val_loss: 0.2675 - val_mae: 0.3877
Epoch 8/69
402788/4

In [ ]:
single_nn.summary()

### Deep Neural Network

The full model is specified as a deep neural network using layers with ReLU activation functions with a linear activation in the output layer. The choice of number of layers was initially kept small due to computational processing constraints.

In [9]:
deep_nn = neural_net(
    x_train,
    y_train,
    n_layers = 10,
    n_hidden_units = n_hidden_units,
    epochs = epochs,
    validation_data = (x_dev, y_dev)
)

Epoch 1/69
402788/402788 [==============================] - 1678s 4ms/step - loss: 0.2217 - mae: 0.3546 - val_loss: 0.1814 - val_mae: 0.3223
Epoch 2/69
402788/402788 [==============================] - 1375s 3ms/step - loss: 0.1995 - mae: 0.3426 - val_loss: 0.1802 - val_mae: 0.3220
Epoch 3/69
402788/402788 [==============================] - 1445s 4ms/step - loss: 0.1979 - mae: 0.3412 - val_loss: 0.1805 - val_mae: 0.3206
Epoch 4/69
402788/402788 [==============================] - 1510s 4ms/step - loss: 0.1969 - mae: 0.3403 - val_loss: 0.1804 - val_mae: 0.3209
Epoch 5/69
402788/402788 [==============================] - 1265s 3ms/step - loss: 0.1963 - mae: 0.3398 - val_loss: 0.1768 - val_mae: 0.3206
Epoch 6/69
402788/402788 [==============================] - 1418s 4ms/step - loss: 0.1959 - mae: 0.3394 - val_loss: 0.1752 - val_mae: 0.3194
Epoch 7/69
402788/402788 [==============================] - 2101s 5ms/step - loss: 0.1956 - mae: 0.3392 - val_loss: 0.1838 - val_mae: 0.3304
Epoch 8/69
40

In [ ]:
deep_nn.summary()

In [ ]:
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
def neural_net_model(
        x_train: Union[tf.Tensor, np.ndarray],
        n_hidden_units: int,
        n_layers: int = 1,
        learning_rate: float = 0.01,
        hidden_activation: str = 'relu',
        output_activation: str = 'linear',
        loss: str = 'mean_squared_error',
    ) -> tf.keras.Model:
    """
    This function creates a neural network model with n_layers hidden layers
    and an output layer using the activations specified.
    """

    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=x_train.shape[1:]))
    
    # Adding hidden layers
    for _ in range(n_layers):
        model.add(
            layers.Dense(
                n_hidden_units,
                kernel_initializer=initializers.he_normal(),
                trainable=True)
            )
        model.add(layers.BatchNormalization())
        model.add(layers.Activation(hidden_activation))
    
    # Adding output layer
    model.add(layers.Dense(
        units=1,
        activation=output_activation,
        kernel_initializer=initializers.glorot_normal(),
        trainable=True))
    
    model.compile(
        optimizer=tf._optimizers.Adam(learning_rate=learning_rate),
        loss=loss
    )
    
    model.fit(x_train, y_train, epochs=10, batch_size=32, verbose=0)

    return model

# Create a wrapper function for the model
model = KerasRegressor(
    build_fn=neural_net_model,
    x_train=x_train,
    y_train = y_train,
    optimizer='Adam',
    loss = 'mean_squared_error',
    epochs=10,
    batch_size=32,
    verbose=0
)

# Define hyperparameter search space
param_dist = {
    'n_hidden_units': [12, 18, 24],
    'n_layers': [5,8,10],
    'learning_rate': [0.001, 0.01, 0.1]
}

random_search = RandomizedSearchCV(
    model,
    param_distributions=param_dist,
    n_iter=10,
    scoring='neg_mean_squared_error',
)
random_search_result = random_search.fit(x_train, y_train)

In [ ]:
random_search_result = random_search.fit(x_train, y_train)

## Results

In [ ]:
nn_dict = {
    'NN - 1 layer': single_nn.history.history['loss'],
    'NN - 5 layers': deep_nn.history.history['loss']
}
baseline_dict = {
    'Random Forest': rfr_mse,
    # 'Random Forest (tuned)': rfr_t_mse,
    'XGBoost': xgb_mse
}

generate_plot(nn_dict, baseline_dict)

### Calculating marginal valuation of environmental attributes

Once we have a model with environmental attributes as features and log of house price as the target variable, we can begin to construct the marginal willingness to pay for environmental attributes, and therefore get a proxy for their value.

The partial derivative of `ln_price` w.r.t an input variable can be evaluated during the backwards propagation stage of the model fitting. By evaluating this derivative at a variety of samples, we can plot a valulation curve for % change in house price resulting from a percentile change in the rank of a house for a given environmental attribute.

In [ ]:
# extract the gradients from the model
points = np.arange(0, 1.05, 0.05)
gradients = calc_partial_grad(
    deep_nn,
    x_train,
    derivative_index,
    points)

# calculate gradients across 20 different points (per attribute)

# plot the derived valuation curves

### Saving models

Here we save the pre-trained models so they can be loaded and used.

In [ ]:
model_dir = cwd / "outputs" / "models"

# save models as tensor objects
deep_nn.save(model_dir / "nn_layers_5")
single_nn.save(model_dir / "nn_layers_1")

# save models as joblib files
joblib.dump(rfr_model, model_dir / "random_forest.joblib")
# joblib.dump(rfr_t_model, model_dir / "random_forest_tuned.joblib")
joblib.dump(xgb_model, model_dir / "xgboost.joblib")